# AI-Orchestrated PowerPoint Report Generator

This notebook uses **your LLM API's agentic capabilities** to automatically generate comprehensive PowerPoint reports from warpage data.

## How It Works

Instead of hardcoding visualizations and slides, this notebook:
1. **Uploads your data** to the LLM API
2. **Sends strategic prompts** to the ReAct agent
3. **Lets the agent autonomously**:
   - Analyze the data
   - Generate visualization code
   - Execute the code
   - Create PowerPoint slides
   - Add charts and formatting

**Key Advantage:** The AI adapts to your data structure, fixes errors automatically, and can suggest additional insights.

**Technology Stack:**
- Your LLM API (ReAct agent + python_coder tool)
- python-pptx (auto-generated)
- matplotlib/seaborn (auto-generated)

**Supports:** Multiple datasets, automatic comparison, adaptive visualizations

## 1. Setup & Configuration

USE THIS AS IT IS, NO NEED TO CHANGE!!

In [ ]:
import httpx
import json
from pathlib import Path
from datetime import datetime
import time

# LLM API Client (from API_examples.ipynb)
class LLMApiClient:
    def __init__(self, base_url: str, timeout: float = 3600.0):
        self.base_url = base_url.rstrip("/")
        self.token = None
        self.timeout = httpx.Timeout(50.0, read=timeout, write=timeout, pool=timeout)

    def _headers(self):
        h = {}
        if self.token:
            h["Authorization"] = f"Bearer {self.token}"
        return h

    def login(self, username: str, password: str):
        r = httpx.post(f"{self.base_url}/api/auth/login", json={
            "username": username, "password": password
        }, timeout=10.0)
        r.raise_for_status()
        data = r.json()
        self.token = data["access_token"]
        return data

    def list_models(self):
        headers = {"Authorization": f"Bearer {self.token}"} if self.token else {}
        r = httpx.get(f"{self.base_url}/v1/models", headers=headers, timeout=10.0)
        r.raise_for_status()
        return r.json()

    def chat_new(self, model: str, user_message: str, agent_type: str = "auto", files: list = None):
        messages = [{"role": "user", "content": user_message}]
        data = {
            "model": model,
            "messages": json.dumps(messages),
            "agent_type": agent_type
        }
        
        files_to_upload = []
        if files:
            for file_path in files:
                f = open(file_path, "rb")
                files_to_upload.append(("files", (Path(file_path).name, f)))
        
        try:
            r = httpx.post(
                f"{self.base_url}/v1/chat/completions",
                data=data,
                files=files_to_upload if files_to_upload else None,
                headers=self._headers(),
                timeout=self.timeout
            )
            r.raise_for_status()
            result = r.json()
            return result["choices"][0]["message"]["content"], result["x_session_id"]
        finally:
            for _, (_, f) in files_to_upload:
                f.close()

    def chat_continue(self, model: str, session_id: str, user_message: str, agent_type: str = "auto", files: list = None):
        messages = [{"role": "user", "content": user_message}]
        data = {
            "model": model,
            "messages": json.dumps(messages),
            "session_id": session_id,
            "agent_type": agent_type
        }
        
        files_to_upload = []
        if files:
            for file_path in files:
                f = open(file_path, "rb")
                files_to_upload.append(("files", (Path(file_path).name, f)))
        
        try:
            r = httpx.post(
                f"{self.base_url}/v1/chat/completions",
                data=data,
                files=files_to_upload if files_to_upload else None,
                headers=self._headers(),
                timeout=self.timeout
            )
            r.raise_for_status()
            result = r.json()
            return result["choices"][0]["message"]["content"], result["x_session_id"]
        finally:
            for _, (_, f) in files_to_upload:
                f.close()

# Configuration
API_BASE_URL = 'http://localhost:1007'
USERNAME = "leesihun"
PASSWORD = "s.hun.lee"

# Initialize client
client = LLMApiClient(API_BASE_URL, timeout=3600.0)
print(f"Client initialized: {API_BASE_URL}")

In [ ]:
# Login and get model
client.login(USERNAME, PASSWORD)
models = client.list_models()
MODEL = models["data"][0]["id"]

print(f"Logged in as: {USERNAME}")
print(f"Using model: {MODEL}")

## 2. Configure Data Files

**Edit this cell** to specify your warpage statistics files:

In [ ]:
# ========================================
# CONFIGURATION: Define your data files
# ========================================

# Option 1: Single file
# stats_paths = [Path(f"data/uploads/{USERNAME}/20251013_stats.json")]

# Option 2: Multiple files for comparison
stats_paths = [
    Path(f"B8_1021_stats.json"),
    Path(f"B8_1027_stats.json"),
]

# Verify files exist
print(f"Configured {len(stats_paths)} data file(s):\n")


for i, path in enumerate(stats_paths, 1):
    if path.exists():
        size_kb = path.stat().st_size / 1024
        print(f"  [{i}] {path.name} ({size_kb:.1f} KB) - OK")
    else:
        print(f"  [{i}] {path.name} - FILE NOT FOUND")

# Convert to strings for API
file_paths_str = [str(p) for p in stats_paths]

## 3. Phase 1: Data Analysis (AI-Driven)

Let the AI agent analyze your data and recommend visualizations.

In [ ]:
analysis_prompt = f"""
I have {len(stats_paths)} warpage measurement dataset(s) attached as JSON files.

The JSON file contains, numerous data, of which correspond to each files.
Under 'files', there are information regarding each PCB.
'min' means minimum z-height, whereas 'max' means maximum.
'range' equals max_minus_min, which is the warpage valeu.
mean, median, std, skewness, and kurtosis are the stats inside a single PCB board.
the PCA values are calculated within each source_pdf. Keep that in mind, and irregularity assessment of PCA should be done by assessing both PCA values.
Now, with these files in mind,

TASK: Comprehensive Data Analysis

Please perform the following analysis:

1. **Calculate Key Statistics:**
   - Total number of measurements across all datasets
   - Overall mean, std, min, max, range
   - Number of outliers by viewing PCA values as shown in pc1, pc2
   - Dataset comparison between different production dates

2. **Identify Visualization Needs:**
   - What trends should be visualized?
   - What comparisons are important?
   - What outliers or anomalies should be highlighted?

3. **Identify outliers:**
   - Find which files are outliers. Give the full file names.
   - Add reasons why those are outliers, considering many aspects.

4. **Return Summary:**
   Provide a clear summary with:
   - Which data is better and why?
   - Any concerns or interesting patterns found

Be specific and data-driven in your recommendations.
"""

print("=" * 80)
print("PHASE 1: DATA ANALYSIS")
print("=" * 80)
print(f"\nUploading {len(stats_paths)} file(s) to AI agent...\n")

start_time = time.time()
analysis_result, session_id = client.chat_new(
    MODEL,
    analysis_prompt,
    agent_type="auto",  # Let agent decide (will likely use python_coder)
    files=file_paths_str
)
elapsed = time.time() - start_time

print(f"\nAnalysis completed in {elapsed:.1f}s\n")
print("=" * 80)
print("AI ANALYSIS RESULT")
print("=" * 80)
from IPython.display import display, Math, Latex
display(Latex(analysis_result))
print("\n" + "=" * 80)

## 4. Phase 2: Generate Visualizations (AI-Driven)

Let the AI agent create all visualizations based on the analysis.

In [ ]:
visualization_prompt = f"""
**PRIORITY: Use your Phase 1 analysis results first.**

In Phase 1, you already analyzed {len(stats_paths)} warpage datasets and loaded all the data.
You identified:
- Outliers via PCA (pc1, pc2) analysis
- Production date comparison results
- Key statistics for all files (mean, std, min, max, range)
- Specific outlier file names and their characteristics

**USE THESE FINDINGS from your Phase 1 analysis - DO NOT re-analyze from scratch.**

The attached files are ONLY for reference if you need to verify or re-check specific raw values.
Your primary data source should be what you already calculated in Phase 1.

TASK: Create High-Quality Visualizations with Outlier Classification (Good vs Bad)

**CRITICAL REQUIREMENT:**
From Phase 1, you identified outliers via PCA (pc1, pc2) analysis.
Now you must classify these outliers as either GOOD QUALITY or BAD QUALITY based on warpage metrics.

**Outlier Quality Classification Logic:**

1. **Identify Spatial Outliers (PCA-based):**
   - Use the PCA outliers you already identified in Phase 1
   - Calculate distance from PCA cluster center: sqrt((pc1 - mean_pc1)^2 + (pc2 - mean_pc2)^2)
   - Outliers = files with distance > 95th percentile (or similar threshold)

2. **Classify Outlier Quality:**
   
   **BAD Quality Outliers (highest priority):**
   - High absolute mean warpage (far from target 0)
   - High standard deviation (high variability)
   - High range (extreme fluctuations)
   - Multiple quality issues combined
   - **These are critical problems requiring immediate action**
   
   **GOOD Quality Outliers (rare but possible):**
   - Spatial outliers but with good metrics:
   - Low std, mean near 0, consistent measurements
   - Unusual PCA position but acceptable warpage
   - **These are acceptable, just unusual**
   
   **Normal (non-outliers):**
   - Within normal PCA cluster
   - Standard quality metrics

**Required Visualizations:**

**1. PCA Outlier Classification** (CRITICAL - save as: pca_outliers_classified.png)
   - Scatter plot: PC1 vs PC2
   - Color coding:
     - Blue dots: Normal samples
     - Orange dots: Good quality outliers (if any exist)
     - RED dots: BAD quality outliers (CRITICAL)
   - Point size: Proportional to warpage severity (larger = worse)
   - Annotations: Label BAD outliers with their file_id or filename
   - Legend: Clearly explain color coding
   - Title: "PCA-Based Outlier Classification: Quality Assessment"

**2. Bad Outlier Details** (save as: bad_outliers_detail.png)
   - Create comparative visualization for BAD outliers:
   - Option A: Bar chart comparing each bad outlier's metrics (mean, std, range) vs dataset average
   - Option B: Small multiples showing each bad outlier's key statistics
   - Include annotations explaining WHY each is bad:
     - "File_XX: High variability (std=87.89 vs avg=79.5)"
     - "File_YY: Extreme mean (mean=-311 vs avg=-297)"
   - Use RED color scheme to emphasize problems

**3. Production Date Comparison** (save as: production_comparison.png)
   - Compare datasets by production date (extract from filenames or metadata)
   - Show multiple metrics:
     - Average mean warpage per date
     - Average std deviation per date
     - Number of bad outliers per date
     - Overall quality score per date
   - Use bar charts or grouped comparisons
   - Highlight which production date has better quality
   - Title: "Production Date Quality Comparison"

**4. Additional Standard Charts** (create as appropriate based on data):

   a. **Temporal Trends** (temporal_trends.png) - if applicable:
      - Line chart: mean values over file sequence
      - Add ±1σ envelope
      - Mark bad outliers with red X markers
   
   b. **Distribution Analysis** (distributions.png):
      - 2x2 grid of histograms: mean, std, skewness, kurtosis
      - Mark positions of bad outliers with red vertical lines
   
   c. **Control Chart** (control_chart.png):
      - Mean values with ±3σ control limits
      - Highlight bad outliers in red
   
   d. **Box Plots** (boxplots.png):
      - Show min/max/range, mean/median, std distributions
      - Overlay bad outlier positions
   
   e. **Correlation Heatmap** (correlation_heatmap.png):
      - Correlations between all metrics
      - Annotate with coefficients

**Technical Requirements:**
- Save all charts to 'temp_charts/' directory (create if doesn't exist)
- Use 300 DPI for all images
- Professional color scheme:
  - Normal: Blue (#1f77b4)
  - Good outliers: Orange (#ff7f0e)
  - BAD outliers: Red (#d62728)
  - Other: Green (#2ca02c), Gray (#7f7f7f)
- Set style: seaborn whitegrid
- Add proper titles, labels, legends
- Use tight_layout() before saving
- Font sizes: Title 14pt, labels 12pt, annotations 10pt

**Packages:**
- matplotlib.pyplot
- seaborn
- pandas
- numpy
- scipy (for distance calculations)
- sklearn.preprocessing.MinMaxScaler (if needed)

**REQUIRED OUTPUT:**
After generating all charts, provide:

1. **List of Generated Files:**
   - pca_outliers_classified.png - [brief description]
   - bad_outliers_detail.png - [brief description]
   - production_comparison.png - [brief description]
   - [other charts].png - [brief descriptions]

2. **Bad Outlier Summary** (for PowerPoint executive slide):
   - List each BAD outlier with:
     - File ID/name
     - Specific reasons (e.g., "High std: 87.89", "Extreme mean: -311.45")
     - Severity ranking (1=worst, 2=2nd worst, etc.)

3. **Production Date Insights:**
   - Which production date is better and why
   - Quality metrics comparison

4. **Key Insights for Executive Summary:**
   - Total bad outliers count
   - Most critical quality issue found
   - Recommendation for immediate action

Print confirmation after EACH chart is saved.
Be thorough in classification - bad outliers need immediate attention!
"""

print("=" * 80)
print("PHASE 2: VISUALIZATION GENERATION WITH OUTLIER CLASSIFICATION")
print("=" * 80)
print(f"\nGenerating professional visualizations with quality assessment...\n")
print("NOTE: Prioritizing Phase 1 analysis results, files available as fallback\n")

start_time = time.time()
viz_result, _ = client.chat_continue(
    MODEL,
    session_id,
    visualization_prompt,
    agent_type="auto",
    files=file_paths_str  # Keep as fallback reference
)
elapsed = time.time() - start_time

print(f"\nVisualization generation completed in {elapsed:.1f}s\n")
print("=" * 80)
print("AI VISUALIZATION RESULT")
print("=" * 80)
display(Latex(viz_result))
print("\n" + "=" * 80)

## 5. Phase 3: PowerPoint Assembly (AI-Driven)

Let the AI agent create the PowerPoint presentation with all charts.

In [ ]:
# Get total file count for the prompt
total_files = 0
for path in stats_paths:
    with open(path, 'r') as f:
        data = json.load(f)
        total_files += len(data.get('files', []))

pptx_prompt = f"""
**PRIORITY: Use your Phase 1 & Phase 2 findings first.**

In Phase 1, you analyzed {len(stats_paths)} warpage datasets and identified:
- Key statistics (total measurements, mean, std, min, max, range)
- Outliers via PCA (pc1, pc2) analysis
- Production date comparison results
- Specific outlier file names and characteristics

In Phase 2, you classified outliers and generated visualizations:
- BAD quality outliers (high mean/std/range - critical issues)
- GOOD quality outliers (unusual PCA but acceptable metrics)
- Created charts in temp_charts/ directory
- Provided bad outlier summary with specific file IDs and reasons

**USE THESE FINDINGS from Phase 1 & 2 - DO NOT re-analyze the raw data files.**

The attached files are ONLY for reference if you need to verify specific values.
Your primary information sources are:
1. Phase 1 analysis results (conversation context)
2. Phase 2 visualization results and bad outlier classifications (conversation context)
3. Generated PNG charts in temp_charts/ directory

TASK: Create a professional PowerPoint presentation using python-pptx with CRITICAL FOCUS on bad quality outliers.

**CONTEXT FROM PREVIOUS PHASES:**
- Phase 1: Identified outliers via PCA analysis and compared production dates
- Phase 2: Classified outliers as GOOD or BAD quality
- Phase 2: Generated visualizations in temp_charts/ directory
- Phase 2: Provided bad outlier summary and production date insights

**CRITICAL REQUIREMENT:**
The FIRST CONTENT SLIDE (Slide 2) must prominently display BAD OUTLIERS as an alert/warning.
Bad outliers require immediate attention and must be visible immediately.

**Presentation Details:**
- Title: "Warpage Analysis Report"
- Subtitle: "Statistical Analysis of {total_files} Measurement Files ({len(stats_paths)} Production Dates)"
- Slide size: 10 x 7.5 inches
- Output filename: Warpage_Report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pptx

**MANDATORY SLIDE STRUCTURE:**

═══════════════════════════════════════════════════════════════════════════════
**Slide 1: Title Slide**
═══════════════════════════════════════════════════════════════════════════════
- Title: "Warpage Analysis Report" (44pt, bold, blue #1f77b4, centered)
- Subtitle: "Statistical Analysis of {total_files} Measurement Files ({len(stats_paths)} Production Dates)" (24pt, centered)
- Date: {datetime.now().strftime('%Y-%m-%d')} (14pt, italic, centered)
- Use blank layout (index 6)

═══════════════════════════════════════════════════════════════════════════════
**Slide 2: EXECUTIVE SUMMARY - QUALITY ALERT** ⚠️ CRITICAL SLIDE
═══════════════════════════════════════════════════════════════════════════════
**This is the MOST IMPORTANT slide - bad outliers must be prominently featured**

**Layout Structure (top to bottom):**

1. **Title** (top of slide, 0.3 inches from top):
   - Text: "Executive Summary - Quality Alert"
   - Font: 32pt, bold, RED (#d62728)
   - Alignment: CENTER

2. **RED ALERT BOX** (1.0 inch from top, 9 inches wide, 0.5 inches from left):
   - Shape: Rectangle with RED fill (#d62728)
   - Border: White, 2pt
   - Height: Auto-size based on content (min 1.5 inches)
   
   **Inside Alert Box (white text on red background):**
   - Header: "⚠ BAD QUALITY OUTLIERS IDENTIFIED" (20pt, bold, white)
   - Subheader: "Immediate Action Required" (14pt, white)
   - **List of Bad Outliers** (extracted from Phase 2 results in conversation context):
     - Each outlier on new line with bullet point
     - Format: "• [File ID/Name]: [Reason 1], [Reason 2]" (12pt, white)
     - Example: "• File_04_B8_1021: High variability (std=87.89), Extreme mean (-311.45)"
     - List ALL bad outliers identified by Phase 2 (from your conversation memory)
   - If no bad outliers: "✓ No critical quality issues detected" (16pt, white)

3. **KEY METRICS CARDS** (below alert box, ~3.0 inches from top):
   - 4 colored metric cards in a row (each 2 inches wide, 1.2 inches tall)
   - Card positions: 0.5, 2.8, 5.1, 7.4 inches from left
   
   **Card 1 (Blue background #1f77b4):**
   - Value: "{total_files}" (28pt, bold, white, centered)
   - Label: "Total Measurements" (11pt, white, centered)
   
   **Card 2 (Green background #2ca02c if winner, Orange #ff7f0e if loser):**
   - Value: "[Production Date from Phase 1 conversation]" (24pt, bold, white, centered)
   - Label: "Better Production Date" (11pt, white, centered)
   
   **Card 3 (Red background #d62728):**
   - Value: "[Bad outlier count from Phase 2 conversation]" (28pt, bold, white, centered)
   - Label: "Bad Outliers" (11pt, white, centered)
   
   **Card 4 (Orange background #ff7f0e):**
   - Value: "[Avg mean from Phase 1 conversation]" (24pt, bold, white, centered)
   - Label: "Avg Mean Warpage" (11pt, white, centered)

4. **PRODUCTION DATE COMPARISON** (below cards, ~4.5 inches from top):
   - Textbox: 0.5 inches from left, 8.5 inches wide
   - Title: "Production Date Analysis:" (16pt, bold, black)
   - Content from Phase 1 analysis (conversation context):
     - "[Date 1]: [Quality assessment]"
     - "[Date 2]: [Quality assessment]"
     - "Winner: [Date] - Reason: [from Phase 1]"
   - Font: 12pt, black

5. **KEY INSIGHTS** (bottom section, ~5.5 inches from top):
   - Textbox: 0.5 inches from left, 8.5 inches wide
   - Title: "Critical Findings:" (14pt, bold, black)
   - Bullet points from Phase 1 & 2 analysis (12pt):
     - Most critical quality issue
     - Overall data quality assessment
     - Recommended immediate actions

═══════════════════════════════════════════════════════════════════════════════
**Slide 3: PCA-Based Outlier Classification**
═══════════════════════════════════════════════════════════════════════════════
- Title: "Outlier Classification: Quality Assessment" (28pt, bold, blue)
- Image: temp_charts/pca_outliers_classified.png (9 inches wide, centered, 1 inch from top of content area)
- Description textbox (below image, 0.5 inch margin):
  - "Color Coding:" (12pt, bold)
  - "• Blue: Normal quality samples"
  - "• Orange: Good outliers (unusual but acceptable)"
  - "• RED: BAD outliers (require immediate action)"
  - Font: 11pt
- **List bad outliers again** (below description):
  - "Critical Files:" (12pt, bold, red)
  - Extract from Phase 2 conversation: list file IDs/names with issues (11pt)

═══════════════════════════════════════════════════════════════════════════════
**Slide 4: Bad Outlier Detailed Analysis**
═══════════════════════════════════════════════════════════════════════════════
- Title: "Detailed Analysis: Bad Quality Outliers" (28pt, bold, red #d62728)
- Image: temp_charts/bad_outliers_detail.png (if exists, 9 inches wide, centered)
- Description textbox:
  - Title: "Why These Samples Failed Quality Standards:" (14pt, bold, black)
  - For EACH bad outlier (from Phase 2 summary in conversation):
    - "• [File ID]: [Detailed explanation]"
    - Include specific metrics (mean, std, range values)
    - Compare to dataset average
  - Font: 11pt
- Action items textbox (bottom of slide):
  - Title: "Required Actions:" (14pt, bold, red)
  - Bullet points:
    - "Root cause analysis for each bad outlier"
    - "Review manufacturing process at time of measurement"
    - "Implement corrective actions"
  - Font: 12pt

═══════════════════════════════════════════════════════════════════════════════
**Slide 5: Production Date Quality Comparison**
═══════════════════════════════════════════════════════════════════════════════
- Title: "Production Date Comparison" (28pt, bold, blue)
- Image: temp_charts/production_comparison.png (9 inches wide, centered)
- Summary textbox (below image):
  - From Phase 1 analysis conversation:
    - "Better Production Date: [Date] - [Reason]"
    - "Quality Metrics Comparison: [Details]"
    - "Recommendation: [Process improvement suggestion]"
  - Font: 12pt

═══════════════════════════════════════════════════════════════════════════════
**Slides 6+: Additional Visualizations (ADAPTIVE)**
═══════════════════════════════════════════════════════════════════════════════
**IMPORTANT: Discover and add slides for ALL remaining charts in temp_charts/**

For each additional PNG file found in temp_charts/ (besides those already used):
- Create a new slide with blank layout
- Add title based on filename (convert underscore to space, title case)
  - Example: "temporal_trends.png" → "Temporal Trends"
- Add image (9 inches wide, centered)
- Add brief description based on chart type:
  - temporal_trends: "Mean warpage values over measurement sequence with variability envelope"
  - distributions: "Statistical distribution analysis across all metrics"
  - control_chart: "Statistical process control with ±3σ limits"
  - boxplots: "Variability analysis showing spread and quartiles"
  - correlation_heatmap: "Metric correlations (values close to ±1 indicate strong relationships)"
  - [other charts]: Provide appropriate description

**Common charts that might exist:**
- temporal_trends.png
- distributions.png
- control_chart.png
- boxplots.png
- correlation_heatmap.png
- summary_table.png
- etc.

═══════════════════════════════════════════════════════════════════════════════
**FINAL SLIDE: Recommendations & Action Items**
═══════════════════════════════════════════════════════════════════════════════
- Title: "Recommendations & Action Items" (32pt, bold, blue)

**Section 1: IMMEDIATE ACTIONS** (0.8 inches from top):
- Header: "🔴 IMMEDIATE ACTIONS REQUIRED" (18pt, bold, red #d62728)
- Bullets (12pt):
  - "Investigate bad outlier files: [list specific files from Phase 2 conversation]"
  - "Root cause analysis for [production date with worse quality from Phase 1]"
  - "Implement corrective actions based on findings"
  - "Quarantine or retest affected samples"

**Section 2: QUALITY IMPROVEMENT** (~2.5 inches from top):
- Header: "📊 Quality Improvement Strategy" (18pt, bold, orange #ff7f0e)
- Bullets (12pt, based on Phase 1 findings from conversation):
  - "Replicate best practices from [better production date]"
  - "Target metrics: Mean near 0, Std < [threshold from best batch]"
  - "Reduce variability through process optimization"
  - "Focus on [specific issue found in Phase 1]"

**Section 3: MONITORING & PREVENTION** (~4.5 inches from top):
- Header: "🔍 Ongoing Monitoring" (18pt, bold, blue #1f77b4)
- Bullets (12pt):
  - "Implement PCA-based outlier detection system"
  - "Set automated alerts for quality deviations (±3σ limits)"
  - "Conduct weekly quality reviews"
  - "Track improvement metrics over time"

**Section 4: SUMMARY** (bottom, ~6.0 inches from top):
- Textbox with gray background (#f0f0f0):
  - "Based on this analysis of {total_files} measurements across {len(stats_paths)} production dates:"
  - "• [Count] bad outliers identified requiring immediate action"
  - "• Production date [winner] shows superior quality"
  - "• Recommended focus: [key issue from Phase 1]"
  - Font: 11pt, black

═══════════════════════════════════════════════════════════════════════════════
**TECHNICAL REQUIREMENTS:**
═══════════════════════════════════════════════════════════════════════════════
- Use blank layout (index 6) for all slides
- RGB colors: Blue (31,119,180), Orange (255,127,14), Green (44,160,44), Red (214,39,40)
- All titles: PP_ALIGN.CENTER
- All textboxes: 0.5 inch margins unless specified
- Image dimensions: 9 inches wide (height auto-scale maintaining aspect ratio)
- Shape fills: Use .fill.solid() then .fill.fore_color.rgb = RGBColor(...)
- Text in shapes: Use separate textbox overlays with white text

**CRITICAL IMPLEMENTATION NOTES:**
1. **PRIORITY**: Extract bad outlier information from Phase 2 results in conversation context (not from files)
2. **PRIORITY**: Extract production date comparison from Phase 1 results in conversation context (not from files)
3. Scan temp_charts/ directory to find all PNG files
4. Adapt slide count based on charts found (don't hardcode exact number)
5. Ensure bad outliers appear prominently on Slides 2, 3, and 4
6. Make RED ALERT BOX on Slide 2 highly visible and impactful
7. Link all insights from Phase 1 and Phase 2 throughout presentation
8. **Files are ONLY for fallback reference - use conversation memory first**

**OUTPUT REQUIREMENTS:**
Print after completion:
- Filename: [full path]
- File size: [KB]
- Total slides: [count]
- Bad outliers highlighted: [list]
- Charts included: [list all PNG files used]

**REMEMBER:**
- Slide 2 is the MOST IMPORTANT - bad outliers must be IMMEDIATELY VISIBLE
- Use information from Phase 1 (analysis_result) and Phase 2 (viz_result) in conversation
- Be thorough - this report drives quality improvement decisions!
"""

print("=" * 80)
print("PHASE 3: POWERPOINT ASSEMBLY WITH OUTLIER FOCUS")
print("=" * 80)
print(f"\nCreating PowerPoint with bad outlier alert as priority...\n")
print("NOTE: Prioritizing Phase 1 & 2 conversation findings, files available as fallback\n")

start_time = time.time()
pptx_result, _ = client.chat_continue(
    MODEL,
    session_id,
    pptx_prompt,
    agent_type="auto",
    files=file_paths_str  # Keep as fallback reference
)
elapsed = time.time() - start_time

print(f"\nPowerPoint creation completed in {elapsed:.1f}s\n")
print("=" * 80)
print("AI POWERPOINT RESULT")
print("=" * 80)
display(Latex(pptx_result))
print("\n" + "=" * 80)

## 6. Summary & Next Steps

In [ ]:
from pathlib import Path
import glob

print("\n" + "=" * 80)
print("REPORT GENERATION COMPLETE!")
print("=" * 80)

# Find generated PowerPoint
pptx_files = sorted(glob.glob("Warpage_Report_*.pptx"), reverse=True)
if pptx_files:
    latest_pptx = pptx_files[0]
    size_kb = Path(latest_pptx).stat().st_size / 1024
    print(f"\nGenerated PowerPoint:")
    print(f"  File: {latest_pptx}")
    print(f"  Size: {size_kb:.2f} KB")
else:
    print("\nWarning: Could not find generated PowerPoint file")

# Check generated charts
temp_charts = Path("temp_charts")
if temp_charts.exists():
    chart_files = list(temp_charts.glob("*.png"))
    print(f"\nGenerated Charts: {len(chart_files)}")
    for chart in sorted(chart_files):
        print(f"  - {chart.name}")

print("\n" + "=" * 80)
print("WHAT THE AI DID")
print("=" * 80)
print("""
Phase 1: Data Analysis
  - Loaded and combined all datasets
  - Calculated key statistics
  - Identified patterns and outliers
  - Recommended visualizations

Phase 2: Visualization Generation
  - Generated 8-9 professional charts
  - Used matplotlib/seaborn with 300 DPI
  - Applied professional color schemes
  - Saved all charts to temp_charts/

Phase 3: PowerPoint Assembly
  - Created 11-12 slide presentation
  - Added title, summary, charts, and recommendations
  - Formatted with professional styling
  - Saved with timestamp

Total Steps: All autonomous via ReAct agent + python_coder tool!
""")

print("=" * 80)
print("NEXT STEPS")
print("=" * 80)
print("""
1. Open the generated .pptx file
2. Review charts and insights
3. Customize branding/colors if needed
4. Add company logo
5. Present to stakeholders

To regenerate with different data:
  - Update stats_paths in Section 2
  - Run all cells again

To cleanup temporary files:
  - Delete temp_charts/ directory
  - Delete old .pptx files
""")

print("=" * 80)

## 7. Optional: Cleanup

Remove temporary chart files if desired.

---

## About This Notebook

This notebook demonstrates the power of **AI-orchestrated automation**:

- **~200 lines of orchestration code** vs 2000+ lines of hardcoded logic
- **Adaptive**: AI adjusts to your data structure
- **Self-healing**: Auto-fixes code errors (max 5 retries)
- **Multi-file support**: Built-in from your API
- **Leverages your infrastructure**: ReAct agent + python_coder tool

**How it works:**
1. You provide strategic prompts
2. ReAct agent reasons about the task
3. python_coder generates code
4. Code executes in sandboxed environment
5. Agent verifies results and iterates if needed

**Result:** Comprehensive PowerPoint report generated autonomously!

---

**Version:** 1.0.0 (AI-Orchestrated)  
**Last Updated:** January 2025